In [1]:
from mmcv import Config
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pycocotools.coco import COCO

In [2]:
model_cfg = \
"vfnet_x101_64x4d_fpn_mdconv_c3-800_giou_4conv1f_adamw_3x_coco"

# load config.
cfg = Config.fromfile(os.path.join("save_dir", model_cfg, "configs.py"))

In [3]:
# checkpoint
checkpoint_path = os.path.join(cfg.work_dir, "best_bbox_mAP_50.pth")

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=cfg.data.workers_per_gpu, dist=False, shuffle=False)

model = build_detector(cfg.model, test_cfg=cfg.get("test_cfg"))
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Use load_from_local loader


In [4]:
output = single_gpu_test(model, data_loader, show_score_thr=0.001)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 0.1 task/s, elapsed: 6525s, ETA:     0s

In [5]:
try:
    prediction_strings = []
    file_names = []
    coco = COCO(cfg.data.test.ann_file)
    imag_ids = coco.getImgIds()

    class_num = 11
    for i, out in enumerate(output):
        prediction_string = ""
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for j in range(class_num):
            for o in out[j]:
                prediction_string += str(j) + " " + str(o[4]) + " " + str(o[0]) + " " + str(o[1]) + " " + str(o[2]) + " " + str(o[3]) + " "

        prediction_strings.append(prediction_string)
        file_names.append(image_info["file_name"])
except:
    prediction_strings = []
    file_names = []
    coco = COCO(cfg.data.test.ann_file)
    imag_ids = coco.getImgIds()

    class_num = 11
    for i, out in enumerate(output):
        prediction_string = ""
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for j in range(class_num):
            for o in out[0][j]:
                prediction_string += str(j) + " " + str(o[4]) + " " + str(o[0]) + " " + str(o[1]) + " " + str(o[2]) + " " + str(o[3]) + " "

        prediction_strings.append(prediction_string)
        file_names.append(image_info["file_name"])
    
submission = pd.DataFrame()
submission["PredictionString"] = prediction_strings
submission["image_id"] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f"submission_{model_cfg}.csv"), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


PredictionString              image_id
0  1 0.30126488 165.33348 75.1192 254.18372 165.0...  batch_01_vt/0021.jpg
1  1 0.31612742 0.62828404 245.44281 305.8191 458...  batch_01_vt/0028.jpg
2  1 0.7332649 72.510414 167.9781 373.03625 379.7...  batch_01_vt/0031.jpg
3  1 0.10324043 303.28363 171.4487 318.9262 177.9...  batch_01_vt/0032.jpg
4  1 0.25827807 389.6873 268.97485 485.45563 345....  batch_01_vt/0070.jpg